# Time-Series Forecasting | Corporación Favorita Grocery Sales
## Section 2: Exploratory Data Analysis
### 1 - Notebook Overview

The goal of this notebook is to conduct an exploratory analysis of the **Pichincha** sample dataset extracted in [Section 1: Data Preparation](/notebooks/01-data-preparation.ipynb) and uncover the underlying patterns of grocery sales in Ecuador. Specifically, this notebook will:

- **Outlier Validation:** Evaluating high-sales peaks identified via z-scores to decide if they represent true market signals or noise.
- **Trend & Seasonality Visualization:** Using total sales line plots and Year-Month heatmaps to identify growth trends, December peaks, and period-specific anomalies.
- **Holiday & Event Impact:** Quantifying the "lift" or "suppression" of demand by merging the holiday calendar and analyzing average sales by day-type.
- **Perishability & Category Dynamics:** Segmenting sales between perishable and non-perishable items to assess volume shares and waste risk—a critical factor for forecast accuracy.
- **Time-Series Diagnostics:**
  - **Autocorrelation:** Identifying how past sales influence future values to inform "lag" feature selection.
  - **Stationarity:** Testing the data's statistical properties to determine if differencing is required for modeling.

By the end of this analysis, we will have a "signal-ready" dataset and a clear understanding of the external factors (holidays, oil prices) that must be incorporated into the forecasting models in Section 3.

### 2 - Import Libraries

In [1]:
# Core libraries
import pandas as pd

# File handling
import os

### 3 - Data Ingestion

In [2]:
# Define local paths to data files
holidays_path = "../data/holidays_events.csv"
items_path = "../data/items.csv"
oil_path = "../data/oil.csv"
stores_path = "../data/stores.csv"
train_path = "../data/train_sample.pkl"
transactions_path = "../data/transactions.csv"

# List of all paths to verify
required_paths = [
    holidays_path, 
    items_path, 
    oil_path, 
    stores_path, 
    train_path,
    transactions_path
]

# Create a list to track any missing files
missing_files = []

for path in required_paths:
    if not os.path.exists(path):
        missing_files.append(path)

if missing_files:
    print("❌ ERROR: Required data files not found in the /data/ folder.")
    print("-" * 50)
    for f in missing_files:
        print(f"Missing: {f}")
    print("-" * 50)
    print("To run this notebook, ensure you have run the Data Preparation notebook")
    print("and placed the Kaggle CSVs in your './data/' directory.")
    print("Link: https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting")
    print("-" * 50)
else:
    print("✅ All required data files detected. Ready to load.")

    # Load datasets
    df_holidays = pd.read_csv(holidays_path)
    df_items = pd.read_csv(items_path)
    df_oil = pd.read_csv(oil_path)
    df_stores = pd.read_csv(stores_path)
    df_train = pd.read_pickle(train_path)
    df_transactions = pd.read_csv(transactions_path)

✅ All required data files detected. Ready to load.


In [13]:
# List of dataframe names
df_names = [
    "df_holidays", 
    "df_items", 
    "df_oil", 
    "df_stores", 
    "df_train", 
    "df_transactions"
]

# Helper function to do a sanity check
def sanity_check(df_names):
    print("SANITY CHECK")
    print("=" * 50)
    for name in df_names:
        if name in globals():
            df = globals()[name]
            print(f"--- {name.upper()} ---")
            # Check shape
            print(f"Shape: {df.shape}")
            # Check memory usage
            memory_gb = df.memory_usage(deep=True).sum() / (1024**3)
            print(f"Memory: {memory_gb:.2f} GB")
            print("\nColumn Data Types:")
            print(df.dtypes)
            # Display first 3 rows
            print("\nPreview:")
            display(df.head(3))
            print("-" * 50)
        else:
            print(f"⚠️ Warning: {name} not found in global namespace.\n")

# Run it
sanity_check(df_names)

SANITY CHECK
--- DF_HOLIDAYS ---
Shape: (350, 6)
Memory: 0.00 GB

Column Data Types:
date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object

Preview:


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False


--------------------------------------------------
--- DF_ITEMS ---
Shape: (4100, 4)
Memory: 0.00 GB

Column Data Types:
item_nbr       int64
family        object
class          int64
perishable     int64
dtype: object

Preview:


,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0


--------------------------------------------------
--- DF_OIL ---
Shape: (1218, 2)
Memory: 0.00 GB

Column Data Types:
date           object
dcoilwtico    float64
dtype: object

Preview:


...

--------------------------------------------------
--- DF_TRAIN ---
Shape: (68537168, 11)
Memory: 6.38 GB

Column Data Types:
date                 datetime64[ns]
id                            int64
store_nbr                     int64
item_nbr                      int64
unit_sales                  float64
onpromotion                  object
year                          int32
month                         int32
day                           int32
day_of_week                   int32
unit_sales_7d_avg           float64
dtype: object

Preview:


,date,id,store_nbr,item_nbr,unit_sales,onpromotion,year,month,day,day_of_week,unit_sales_7d_avg
0,2013-05-09,5329375,1,96995,1.0,False,2013,5,9,3,1.000000
1,2013-05-10,0,1,96995,0.0,0,2013,5,10,4,0.500000
2,2013-05-11,0,1,96995,0.0,0,2013,5,11,5,0.333333


--------------------------------------------------
--- DF_TRANSACTIONS ---
Shape: (83488, 3)
Memory: 0.01 GB

Column Data Types:
date            object
store_nbr        int64
transactions     int64
dtype: object

Preview:


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358


--------------------------------------------------


### 4 - Data Validation

#### 4.1 Confirming Data Types

In [15]:
# Confirm date columns are in datetime format
df_holidays['date']     = pd.to_datetime(df_holidays['date'])
df_oil['date']          = pd.to_datetime(df_oil['date'])
df_train['date']        = pd.to_datetime(df_train['date'])
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

# Convert integer columns to int32 for alignment and to save memory
df_items[['item_nbr', 'class']] = df_items[['item_nbr', 'class']].astype('int32')
df_stores[['store_nbr', 'cluster']] = df_stores[['store_nbr', 'cluster']].astype('int32')
df_train[['id', 'store_nbr', 'item_nbr']] = df_train[['id', 'store_nbr', 'item_nbr']].astype('int32')
df_transactions[['store_nbr', 'transactions']] = df_transactions[['store_nbr', 'transactions']].astype('int32')

# Convert 'onpromotion' column to boolean
df_items['perishable'] = df_items['perishable'].astype(bool)
df_train['onpromotion'] = df_train['onpromotion'].astype(bool)

#### 4.2 Checking for Missing Data

In [16]:
# Helper function to check missing data
def check_missing(df_names):
    print("MISSING VALUES REPORT")
    print("=" * 50)
    for name in df_names:
        if name in globals():
            df = globals()[name]
            print(f"--- {name.upper()} ---")
            display(df.isnull().sum())
            print("-" * 50)
        else:
            print(f"⚠️ Warning: {name} not found in global namespace.\n")

# Run it
check_missing(df_names)

MISSING VALUES REPORT
--- DF_HOLIDAYS ---


date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

--------------------------------------------------
--- DF_ITEMS ---


item_nbr      0
family        0
class         0
perishable    0
dtype: int64

--------------------------------------------------
--- DF_OIL ---


...

--------------------------------------------------
--- DF_TRAIN ---


date                 0
id                   0
store_nbr            0
item_nbr             0
unit_sales           0
onpromotion          0
year                 0
month                0
day                  0
day_of_week          0
unit_sales_7d_avg    0
dtype: int64

--------------------------------------------------
--- DF_TRANSACTIONS ---


date            0
store_nbr       0
transactions    0
dtype: int64

--------------------------------------------------


The `df_oil` dataset contains **43 missing values** in the `dcoilwtico` column (presumably due to weekends and holidays). While handling these is outside the current scope of this exercise, in a production environment, these should be addressed using `.ffill()` and `.bfill()` to maintain a continuous price signal without changing the column’s numerical data type.

#### 4.3 Handling Outliers

It is essential to account for unusually high sales spikes that may not reflect normal demand patterns. These extreme observations can arise from promotions, special events, or simple data inconsistencies, and they can distort both exploratory analysis and forecasting models if left untreated. To flag these anomalies, I’ll examine the distribution of sales and use **Z‑scores** to identify values that sit far outside the typical range for each store or item.

In [18]:
# Group by store_nbr and item_nbr to calculate mean and std dev
group_stats = df_train.groupby(['store_nbr', 'item_nbr'])['unit_sales']

# Compute mean and standard deviation for each store-item group
df_train['z_score_mean'] = group_stats.transform('mean')
df_train['z_score_std'] = group_stats.transform('std')

# Calculate Z-score for unit_sales in a new column "z_score"
df_train['z_score'] = (df_train['unit_sales'] - df_train['z_score_mean']) / (df_train['z_score_std'].replace(0, 1))

# Define threshold for outliers (e.g., Z-score > 5)
outliers = df_train[df_train['z_score'] > 5]

# Print summary
print(f"Number of outliers detected: {len(outliers)}\n")
display(outliers.head())

Number of outliers detected: 719515



,date,id,store_nbr,item_nbr,unit_sales,onpromotion,year,month,day,day_of_week,unit_sales_7d_avg,z_score_mean,z_score_std,z_score
0,2013-05-09,5329375,1,96995,1.0,False,2013,5,9,3,1.000000,0.00692,0.123238,8.058245
165,2013-10-21,12808797,1,96995,2.0,False,2013,10,21,0,0.285714,0.00692,0.123238,16.172645
214,2013-12-09,15187011,1,96995,2.0,False,2013,12,9,0,0.285714,0.00692,0.123238,16.172645
226,2013-12-21,15798301,1,96995,2.0,False,2013,12,21,5,0.285714,0.00692,0.123238,16.172645
1444,2017-04-22,113206073,1,96995,3.0,False,2017,4,22,5,0.428571,0.00692,0.123238,24.287044


In [19]:
# Create a temporary series for Z-scores to keep df_train clean
group_stats = df_train.groupby(['store_nbr', 'item_nbr'])['unit_sales']

# Calculate using transform (Vectorized for speed)
m = group_stats.transform('mean')
s = group_stats.transform('std')

# Calculate Z-score and create the outlier dataframe
# We use .copy() so we don't accidentally modify the original data
z_scores = (df_train['unit_sales'] - m) / s.replace(0, 1)
df_outliers = df_train[z_scores > 5].copy()
df_outliers['z_score'] = z_scores[z_scores > 5]

print(f"Total rows in Train: {len(df_train)}")
print(f"Number of outliers (Z > 5): {len(df_outliers)}")
print(f"Percentage of data flagged: {(len(df_outliers) / len(df_train)) * 100:.2f}%")

Total rows in Train: 68537168
Number of outliers (Z > 5): 719515
Percentage of data flagged: 1.05%
